![Command](Command.png)

1. A classe __Invoker__ é responsável por iniciar os pedidos. Essa classe deve ter um campo para armazenar a referência para um objeto comando. O remetente aciona aquele comando ao invés de enviar o pedido diretamente para o destinatário. Observe que o remetente não é responsável por criar o objeto comando. Geralmente ele é pré criado através de um construtor do cliente.

2. A interface __Command__ geralmente declara apenas um único método para executar o comando.

3. __ConcreteCommand__ implementam vários tipos de pedidos. Um comando concreto não deve realizar o trabalho por conta própria, mas passar a chamada para um dos objetos da lógica do negócio. Contudo, para simplificar o código, essas classes podem ser fundidas.

   Os parâmetros necessários para executar um método em um objeto destinatário podem ser declarados como campos no comando concreto. Você pode tornar os objetos comando imutáveis ao permitir que apenas inicializem esses campos através do construtor.

4. A classe __Receiver__ contém a lógica do negócio. Quase qualquer objeto pode servir como um destinatário. A maioria dos comandos apenas lida com os detalhes de como um pedido é passado para o destinatário, enquanto que o destinatário em si executa o verdadeiro trabalho.

5. O __Client__ cria e configura objetos comando concretos. O cliente deve passar todos os parâmetros do pedido, incluindo uma instância do destinatário, para o construtor do comando. Após isso, o comando resultante pode ser associado com um ou múltiplos destinatários.

In [1]:
from __future__ import annotations
from abc import ABCMeta, abstractmethod

In [2]:
class Light:
    """ Receiver """
    
    def __init__(self, name: str, room_name: str) -> None:
        self._name = name
        self._room_name = room_name
        self._color = 'Default'
        self._stage = False

    def on(self) -> None:
        if not self._stage:
            self._stage = True
            print(f'Ligth {self._name} in {self._room_name} is now ON')

    def off(self) -> None:
        if self._stage:
            self._stage = False
            print(f'Ligth {self._name} in {self._room_name} is now OFF')

    def change_color(self, color) -> None:
        self._color = color
        print(f'Ligth {self._name} in {self._room_name} is now {self._color}')


In [3]:
class ICommand(metaclass = ABCMeta):
    """" ICommand """
    
    @abstractmethod
    def do(self) -> None: pass

    @abstractmethod
    def undo(self) -> None: pass

In [9]:
class LightoOnCommand(ICommand):
    """" Command """

    def __init__(self, light: Light) -> None:
        self.light = light

    def do(self) -> None:
        self.light.on()

    def undo(self) -> None:
        self.light.off()

class LightoChangeColorCommand(ICommand):
    """" Command """

    def __init__(self, light: Light, color: str) -> None:
        self.light = light
        self.old_color = self.light._color
        self.new_color = color

    def do(self) -> None:
        self.light.change_color(self.new_color)

    def undo(self) -> None:
        self.light.change_color(self.old_color)

In [10]:
class RemoteController:
    """ Invoker """
    
    def __init__(self):
        self._buttons: dict[str, ICommand] = {}

    def add_button_command(self, name: str, command: ICommand) -> None:
        if name not in self._buttons: 
            self._buttons[name] = command

    def do_button_command(self, name: str) -> None:
        if name in self._buttons:
            self._buttons[name].do()

    def undo_button_command(self, name: str) -> None:
        if name in self._buttons:
            self._buttons[name].undo()

In [12]:
if __name__ == "__main__":
    """ CLient """

    bedroom_light = Light('Bedroom Light', 'Bedroom') # Receiver

    bedroom_light_OnOff = LightoOnCommand(light=bedroom_light) # Command
    
    bedroom_light_Collor = LightoChangeColorCommand(light=bedroom_light, color='Red')

    controller = RemoteController() # Invoker
    
    controller.add_button_command('Bedroom Light', bedroom_light_OnOff)
    controller.do_button_command('Bedroom Light')
    controller.undo_button_command('Bedroom Light')

    print()

    controller.add_button_command('Bedroom Light Color', bedroom_light_Collor)
    controller.do_button_command('Bedroom Light Color')
    controller.undo_button_command('Bedroom Light Color')

Ligth Bedroom Light in Bedroom is now ON
Ligth Bedroom Light in Bedroom is now OFF

Ligth Bedroom Light in Bedroom is now Red
Ligth Bedroom Light in Bedroom is now Default
